<a href="https://colab.research.google.com/github/ArthurCTLin/Workbook/blob/main/ML_Course/HW2/Phoneme_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework 2 from [***Machine Learning***](https://speech.ee.ntu.edu.tw/~hylee/ml/2021-spring.html)
* **Professor :** Hung-Yi, Lee
* **Objectives :**
  * Basic understanding of Speech recognition
  * Solve a classification task with DNN
  * Get familiar with PyTorch.
* **Description of HW :** https://speech.ee.ntu.edu.tw/~hylee/ml/ml2021-course-data/hw/HW02/HW02.pdf
* **Kaggle Link :** https://www.kaggle.com/c/ml2021spring-hw2/

### **Download Data**
Download data from google drive, then unzip it.

* `train_11.npy`: training data
* `train_label_11.npy`: training label
* `test_11.npy`:  testing data

**Notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**

In [ ]:
!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
!unzip data.zip
!ls 

Downloading...
From: https://drive.google.com/uc?id=1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR
To: /content/data.zip
100% 372M/372M [00:07<00:00, 46.6MB/s]
Archive:  data.zip
   creating: timit_11/
  inflating: timit_11/train_11.npy   
  inflating: timit_11/test_11.npy    
  inflating: timit_11/train_label_11.npy  
data.zip  sample_data  timit_11


### **Import Package**


In [ ]:
# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# For data preprocess
import numpy as np
import pandas as pd
import csv
import os

# For plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

# Make the results of every experiments are reproducible : https://www.itread01.com/content/1550034200.html
myseed = 0  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True 
torch.backends.cudnn.benchmark = False # https://stackoverflow.com/questions/58961768/set-torch-backends-cudnn-benchmark-true-or-not
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

## **Preparing Data**
* Load Data
* Construct Dataset
* Split Dataset
* DataLoader
* Clean up unnecessary variables


### **Load Data**
Load the training and testing data from the `.npy` file (NumPy array).

In [ ]:
print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


### **Construct Dataset**




In [ ]:
class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None
        self.dim = self.data.shape[1]

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)

### **Split Dataset**
Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [ ]:
VAL_RATIO = 0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (983945, 429)
Size of validation set: (245987, 429)


### **DataLoader**
Create a data loader from a given `Dataset` into batches.

In [ ]:
BATCH_SIZE = 64

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

### **Clean Up Unnecessary Variables**
Cleanup the unneeded variables to save memory.

***Notes :*** if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab

In [ ]:
import gc #Garbage Collection

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

100

## **Create Model**
* Define Model
* Hyperparameters

### **Define Model**
Define architecture of DNN model

In [ ]:
class Classifier(nn.Module):
    def __init__(self, input_dim):
        super(Classifier, self).__init__()
        self.layer1 = nn.Linear(input_dim, 1024)
        self.layer2 = nn.Linear(1024, 512)
        self.layer3 = nn.Linear(512, 512)
        self.layer4 = nn.Linear(512, 256)
        self.layer5 = nn.Linear(256, 128)
        self.layer6 = nn.Linear(256, 128)
        self.layer7 = nn.Linear(128, 128)
        self.layer8 = nn.Linear(128, 64)
        self.out = nn.Linear(128, 39) 

        self.bn_1 = nn.BatchNorm1d(1024)
        self.bn_2 = nn.BatchNorm1d(512)
        self.bn_3 = nn.BatchNorm1d(512)
        self.bn_4 = nn.BatchNorm1d(256)
        self.bn_5 = nn.BatchNorm1d(128)
        self.bn_7 = nn.BatchNorm1d(128)
        self.act_fn = nn.ReLU()
        self.act_LeakyReLU = nn.LeakyReLU(0.1)
        self.act_ELU = nn.ELU()
        self.dropout = nn.Dropout(0.3)
    

    def forward(self, x):
        x = self.layer1(x)
        x = self.bn_1(x)
        x = self.dropout(x)
        #x = self.act_ELU(x)
        x = self.act_LeakyReLU(x)
        #x = self.act_fn(x)
     
        x = self.layer2(x)
        #x = self.bn_2(x)
        x = self.dropout(x)
        #x = self.act_ELU(x)
        x = self.act_LeakyReLU(x)
        #x = self.act_fn(x)

        x = self.layer3(x)
        #x = self.bn_3(x)
        x = self.dropout(x)
        #x = self.act_ELU(x)
        x = self.act_LeakyReLU(x)
        #x = self.act_fn(x)

        x = self.layer4(x)
        x = self.dropout(x)
        #x = self.act_ELU(x)
        x = self.act_LeakyReLU(x)
        #x = self.act_fn(x)

        x = self.layer5(x)
        #x = self.bn_5(x)
        x = self.dropout(x)
        #x = self.act_ELU(x)
        x = self.act_LeakyReLU(x)
        #x = self.act_fn(x)
        
        #x = self.layer7(x)
        #x = self.bn_7(x)
        #x = self.dropout(x)
        #x = self.act_ELU(x)
        #x = self.act_LeakyReLU(x)
        #x = self.act_fn(x)
        
        x = self.out(x)
        
        return x

### **Set Hyper-parameters**

In [ ]:
# get device 
device = get_device()
print(f'DEVICE: {device}')

# training hyper-parameters
num_epoch = 200           # number of training epoch
learning_rate = 0.0001       # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier(train_loader.dataset.dim).to(device)
model.load_state_dict(torch.load("./HW2_28_model.ckpt"))
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

DEVICE: cpu


## **Training & Testing**


### **Start Training**

In [ ]:
best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


### **Testing**
Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier(train_loader.dataset.dim).to(device)
model.load_state_dict(torch.load(model_path))

#### **Make prediction**

In [ ]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

#### **Write prediction to a CSV file**

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))

## **Reference**
* Source code provided by Course TA : https://colab.research.google.com/github/ga642381/ML2021-Spring/blob/main/HW02/HW02-2.ipynb
* Speech Recognition : https://speech.ee.ntu.edu.tw/~tlkagk/courses/DLHLP20/ASR%20(v12).pdf
* Automatic Speech Recognition : https://ithelp.ithome.com.tw/articles/10195763